In [264]:
import params
from google.cloud import datastore, storage, logging
import time
import pickle
import hashlib
import sys
import time
import multiprocessing as mp
import sys
import dataAck
import curveTreeDB


In [288]:
def getPertinentDataModels(db):
    while True:
        try:
            datastore_client = datastore.Client('money-maker-1236')
            query = datastore_client.query(kind=db)
#             query.add_filter("numberOfPredictors", "=", 2)
            query.add_filter("IS_PROFITABILITY", '>', 0.45) #-0.05
            retrievedModels = list(query.fetch())
            return retrievedModels
        except:
            print("DATA SOURCE RETRIEVAL ERROR:", str(sys.exc_info()))
            return 

In [289]:
allValidModels = getPertinentDataModels(params.curveModels)

In [290]:
quickModels = []
predLength = {}
for mod in allValidModels:
#     if mod["predictionLength"] < 10:
#         continue
    if mod["predictionLength"] not in predLength:
        predLength[mod["predictionLength"]] = []
    predLength[mod["predictionLength"]].append(abs(mod["OOS_BETA"] - mod["IS_BETA"]))
#     print(mod["IS_BETA"], mod["OOS_BETA"], mod["predictionLength"])
    quickModels.append(mod)

# for item in sorted(predLength):
#     print(item, sum(predLength[item])/len(predLength[item]))

In [291]:
len(quickModels)

540

In [292]:
import pandas as pd
cleanedModels = []
seenTickers = {}
seenPredictors = {}
seenAlpha = {}
tickerSharpe = {}

for item in quickModels:
#     del item["model"]
#     try:
#         del item["series"]
#     except:
#         pass
#     try:
#         print(item["IS_PROFITABILITY"], item["lastXDays"], item["IS_BETA"], item["OOS_BETA"])
#     except:
#         continue
#     if item["IS_SHARPE DIFFERENCE SLIPPAGE"] < 0.0:
#         continue
    
#     del item["ticker"]
#     if item["IS_PROFITABILITY"] < 0.51 or item["IS_BETA"] > 0.1 or item["IS_ACTIVITY"] < 0.8 or item["IS_SHARPE"] < 1.0:
#         continue

#     if item["OOS_ALPHA"] > 0:
#         continue

#     if item["maxDistance"] > 1.0:
#         continue
    
#     if item["IS_25TH PROFITABILITY 45"] < 0.43:
#         continue
    
#     if item["lookbackDistance"] < 22:
#         continue
        
#     if item["minConfidence"] <= 0.01:
#         continue

    if item["IS_MIN PROFITABILITY 45"] < 0.3:
        continue
        
    if item["IS_RAW BETA"] > 0.3:
        continue
    
#     if item["IS_PROFITABILITY DIFFERENCE"] < 0:
#         continue
    

    if "series" in item:
        item["DAYS BACK"] = int(item["series"].split(",")[1])
        item["SMOOTHING"] = int(item["series"].split(",")[4][:-1])
    
    if "numberOfPredictors" not in item:
        item["numberOfPredictors"] = 0
        
    item["DISTANCE PER POINT"] = item["maxDistance"]/ item["lookbackDistance"]
    
    if item["numberOfPredictors"] not in seenPredictors:
        seenPredictors[item["numberOfPredictors"]] = {"better":0, "greaterThanZero":0, "bad":0, "total":0}
        seenAlpha[item["numberOfPredictors"]] = {"greaterThanZero":0, "total":0}
    
    seenPredictors[item["numberOfPredictors"]]["total"] += 1
    seenAlpha[item["numberOfPredictors"]]["total"] += 1
    if item["OOS_SHARPE"] > item["OOS_FACTOR SHARPE"]:
#         print(item["ticker"], item["OOS_SHARPE"], item["OOS_FACTOR SHARPE"] )
        seenPredictors[item["numberOfPredictors"]]["better"] += 1
    if item["OOS_SHARPE"] > 0:
        seenPredictors[item["numberOfPredictors"]]["greaterThanZero"] += 1
    if item["OOS_SHARPE"] < 0 and item["OOS_FACTOR SHARPE"] > 0:
        seenPredictors[item["numberOfPredictors"]]["bad"] += 1
    if item["OOS_ALPHA"] > 0:
        seenAlpha[item["numberOfPredictors"]]["greaterThanZero"] += 1
    
    print("ticker", item["ticker"])
    print(item["series"])
    print("NUMBER OF PREDICTORS", item["numberOfPredictors"])
    print("SHARPE", item["IS_SHARPE"], item["OOS_SHARPE"], item["OOS_FACTOR SHARPE"])
    print("RAW BETA", item["IS_RAW BETA"], item["OOS_RAW BETA"])
    print("BETA", item["IS_BETA"], item["OOS_BETA"])
    print("ALPHA", item["IS_ALPHA"], item["OOS_ALPHA"])
    print("PROFITABILITY", item["IS_PROFITABILITY"])
    print("PROFITABILITY DIFFERENCE", item["IS_PROFITABILITY DIFFERENCE"])
    print("SHARPE DIFFERENCE", item["IS_SHARPE DIFFERENCE"])
    print("DISTANCE PER POINT", item["lookbackDistance"], item["DISTANCE PER POINT"])
    print("**********")
        
        
        
    if item["ticker"] not in seenTickers:
        seenTickers[item["ticker"]] = 0
    if item["ticker"] not in tickerSharpe:
        tickerSharpe[item["ticker"]] = item["OOS_FACTOR SHARPE"]
    seenTickers[item["ticker"]] += 1
    itemKeys= list(item.keys())
    for key in itemKeys:
        if key.startswith("OOS") and key != "OOS_ALPHA" and key != "OOS_PROFITABILITY DIFFERENCE" and key != "OOS_FACTOR SHARPE" and key != "OOS_RAW BETA" and key != "OOS_PROFITABILITY" and key != "OOS_SHARPE DIFFERENCE SLIPPAGE" and key != "OOS_SHARPE" and key != "OOS_BETA" and key != "OOS_ANNUALIZED RETURN":
            del item[key]
    cleanedModels.append(item)
print(seenTickers)
print(tickerSharpe)
for predictors in sorted(seenPredictors):
    print(predictors, seenPredictors[predictors]["total"],\
          1.0 - seenPredictors[predictors]["bad"]/seenPredictors[predictors]["total"],\
          seenPredictors[predictors]["better"]/seenPredictors[predictors]["total"],\
          seenPredictors[predictors]["greaterThanZero"]/seenPredictors[predictors]["total"])
    
    print("ALPHA", predictors, seenAlpha[predictors]["greaterThanZero"]/seenAlpha[predictors]["total"])
    
    
    
    
    

ticker XOP
('EEM', 5, None, 17, 4)
NUMBER OF PREDICTORS 0
SHARPE 0.7522309938014463 1.1149487139994592 -0.6341368312060983
RAW BETA 0.18530812324930038 0.13750000000000032
BETA 0.1340544207541168 0.09029904940101452
ALPHA 0.12836974754246466 0.15954439084936525
PROFITABILITY 0.4719222462203024
PROFITABILITY DIFFERENCE -0.04211663066954646
SHARPE DIFFERENCE 0.4723012194383423
DISTANCE PER POINT 30 0.045455
**********
ticker IJR
('XHB', 41, None, 17, 5)
NUMBER OF PREDICTORS 0
SHARPE 1.072145308807152 -1.1936233105253558 0.7713856355961505
RAW BETA 0.2160310336272811 0.12600806451612917
BETA 0.10398195951286848 0.19566821307747417
ALPHA 0.11874068808090478 -0.0848589124706099
PROFITABILITY 0.4777542372881356
PROFITABILITY DIFFERENCE -0.05720338983050843
SHARPE DIFFERENCE 0.15952061377456916
DISTANCE PER POINT 30 0.045455
**********
ticker IWN
('SLV', 28, None, None, 1)
NUMBER OF PREDICTORS 0
SHARPE 0.7705303692104675 1.210772795886271 0.8222003731353329
RAW BETA 0.22313250208176455 0.5433

In [293]:
len(cleanedModels)

313

In [121]:
cm = pd.DataFrame(cleanedModels)
for col in cm.columns.values:
    
#     try:
    print(col)
    print("____________________")
    for target in ["OOS_PROFITABILITY", "OOS_SHARPE", "OOS_ALPHA", "OOS_PROFITABILITY DIFFERENCE"]:
        groups = cm.groupby([pd.cut(cm[col].values, 5)])[target]
        print(groups.mean(), groups.size())
    print("**********")
        
#         groupsSharpe = cm.groupby([col])["OOS_PROFITABILITY"].mean()
#         if len(groupsSharpe) < 20:
#             print(groupsSharpe)
#             print(cm.groupby([col])["OOS_SHARPE DIFFERENCE SLIPPAGE"].mean())
#             print(cm.groupby([col])["OOS_SHARPE"].mean())
#             print(cm.groupby([col])["OOS_ALPHA"].mean())
#         print(col, "OOS_PROFITABILITY")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_PROFITABILITY"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
#         print(col, "OOS_SHARPE DIFFERENCE SLIPPAGE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_SHARPE DIFFERENCE SLIPPAGE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_SHARPE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_SHARPE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_ALPHA")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_ALPHA"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_PROFITABILITY DIFFERENCE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_PROFITABILITY DIFFERENCE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_BETA")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_BETA"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_ANNUALIZED RETURN")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_ANNUALIZED RETURN"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
        
#         print(col, "OOS_FACTOR SHARPE")
#         xVals = cm[[col]].values
#         yVals = cm[["OOS_FACTOR SHARPE"]].values
#         plt.plot(xVals, yVals, 'ro')
#         plt.show()
#         plt.draw()
#     except:
#         continue
    



DAYS BACK
____________________
(0.957, 9.6]    0.507055
(9.6, 18.2]     0.499639
(18.2, 26.8]    0.515873
(26.8, 35.4]    0.502834
(35.4, 44.0]    0.491667
Name: OOS_PROFITABILITY, dtype: float64 (0.957, 9.6]     9
(9.6, 18.2]     11
(18.2, 26.8]     6
(26.8, 35.4]     7
(35.4, 44.0]    10
Name: OOS_PROFITABILITY, dtype: int64
(0.957, 9.6]    0.461811
(9.6, 18.2]     0.374459
(18.2, 26.8]    0.983136
(26.8, 35.4]    0.295170
(35.4, 44.0]    0.001133
Name: OOS_SHARPE, dtype: float64 (0.957, 9.6]     9
(9.6, 18.2]     11
(18.2, 26.8]     6
(26.8, 35.4]     7
(35.4, 44.0]    10
Name: OOS_SHARPE, dtype: int64
(0.957, 9.6]    0.012005
(9.6, 18.2]     0.023881
(18.2, 26.8]    0.027036
(26.8, 35.4]    0.044812
(35.4, 44.0]   -0.007127
Name: OOS_ALPHA, dtype: float64 (0.957, 9.6]     9
(9.6, 18.2]     11
(18.2, 26.8]     6
(26.8, 35.4]     7
(35.4, 44.0]    10
Name: OOS_ALPHA, dtype: int64
(0.957, 9.6]   -0.014109
(9.6, 18.2]    -0.031025
(18.2, 26.8]   -0.038360
(26.8, 35.4]   -0.028345
(35.4

TypeError: can't concat bytes to float